# Handout 15

# **Concorrência e Threads**

## **1. Introdução às Threads**

Threads são unidades básicas de execução dentro de um processo. Diferentemente dos processos, que possuem espaços de memória independentes, múltiplas threads dentro do mesmo processo compartilham o mesmo espaço de memória, permitindo comunicação e compartilhamento de dados de forma eficiente.

**Vantagens das Threads:**
- **Desempenho:** Permitem a execução paralela de tarefas, aproveitando melhor os recursos de sistemas multicore.
- **Compartilhamento de Recursos:** Facilitam a comunicação entre tarefas, já que compartilham o mesmo espaço de memória.
- **Responsividade:** Melhoram a responsividade de aplicações, permitindo que tarefas de longa duração sejam executadas em segundo plano.

---

## **2. Criando e Gerenciando Threads com POSIX Threads (pthreads)**

A biblioteca POSIX Threads (pthreads) fornece uma API padrão para criação e gerenciamento de threads em sistemas Unix-like.

### **2.1. Criando uma Thread**

Para criar uma thread, utiliza-se a função `pthread_create`:

```c
#include <pthread.h>

int pthread_create(pthread_t *thread, const pthread_attr_t *attr, void *(*start_routine)(void *), void *arg);
